# Context
John Doe remarked in [#AP1432](http://www.example.com/justalink) that there may be too much code in our application that isn't used at all. Before migrating the application to the new platform, we have to analyze which parts of the system are still in use and which are not.

# Idea
To understand how much code isn't used, we recorded the executed code in production with the coverage tool [JaCoCo](http://www.jacoco.org/). The measurement took place between 21st Oct 2017 and 27st Oct 2017. The results were exported into a CSV file using the JaCoCo command line tool with the following command:

```bash
java -jar jacococli.jar report "C:\Temp\jacoco.exec" --classfiles \
C:\dev\repos\buschmais-spring-petclinic\target\classes --csv jacoco.csv
```

The CSV file contains all lines of code that were passed through during the measurement's time span.

In [1]:
%%classpath add mvn
tech.tablesaw tablesaw-beakerx 0.24.5
tech.tablesaw tablesaw-jsplot 0.24.5

In [2]:
%import tech.tablesaw.api.*
%import tech.tablesaw.columns.*
%import static tech.tablesaw.aggregate.AggregateFunctions.*

tech.tablesaw.beakerx.TablesawDisplayer.register()

OutputCell.HIDDEN

In [3]:
%%bash
wget https://raw.githubusercontent.com/feststelltaste/software-analytics/master/demos/dataset/jacoco.csv

In [4]:
coverage = Table.read().csv("jacoco.csv")
coverage.first(5)

We just take the only relevant data `LINES_COVERED` as well as the information about the packages in `PACKAGE` as well as the `CLASS` columns.

In [5]:
coverage = coverage.retainColumns("PACKAGE", "CLASS", "LINE_COVERED")
coverage.first(5)

# Analysis
It was stated that whole packages wouldn't be needed anymore and that they could be safely removed. Therefore, we sum up the coverage data per class for each package and calculate the coverage ratio for each package.

In [6]:
per_package = coverage.summarize("LINE_COVERED", sum).by("PACKAGE");
per_package

# Visualisation
We plot the data for the coverage ratio to get a brief overview of the result.

In [7]:
def bars = new CategoryBars(
    value: per_package.column('Sum [LINE_COVERED]').asList())
new CategoryPlot(
    categoryNames: per_package.column('PACKAGE').asList(),
    orientation: PlotOrientationType.HORIZONTAL) << bars

# Conclusion 
The JDBC package `org.springframework.samples.petclinic.repository.jdbc` isn't used at all and can be left out safely when migrating to the new platform.